In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer, TextStreamer
import time
import json
import re

/home/ntlpt59/main/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "LiquidAI/LFM2-350M"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="bfloat16",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [146]:
def get_weather(city: str) -> str:
    """Gets the current weather for a given city.
    
    Args:
        city: The name of the city to get weather for.
        
    Returns:
        A string describing the current weather conditions.

    """
    return f"It's 22°C and sunny in {city} with 55% humidity and 8 km/h winds."

def read(path: str) -> str:
    """Reads a file from the local filesystem. You can access any file directly by using this tool.
  Assume this tool is able to read all files on the machine. If the User provides a path to a file assume that path is valid. It is okay to read
  a file that does not exist; an error will be returned.

    Args:
        path: The file path to read from.

    Returns:
        The file contents as a string.
    """
    try:
        with open(path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: File not found: {path}"
    except IOError as e:
        return f"Error reading file {path}: {e}"


def write_file(path: str, content: str) -> str:
    """Writes content to a file.

    Args:
        path: The file path to write to.
        content: The content to write to the file.

    Returns:
        A success or error message.
    """
    try:
        with open(path, 'w', encoding='utf-8') as file:
            file.write(content)
        return f"Successfully wrote to {path}"
    except IOError as e:
        return f"Error writing to file {path}: {e}"

In [147]:
import ast
import re

def parse_tool_calls(assistant):
    """Parse tool calls from string format to structured dict."""
    
    if '<|tool_call_start|>' in assistant:
        tool_message = assistant.split("<|tool_call_end|>")[0].replace("<|tool_call_start|>","")
    else:
        tool_message = ""
        
    # Remove the outer brackets if present
    tool_message = tool_message.strip()
    if tool_message.startswith('[') and tool_message.endswith(']'):
        tool_message = tool_message[1:-1]
    
    # Split multiple function calls
    # Simple approach: split by ), and add ) back except for last one
    calls = []
    
    # Find all function calls using regex
    pattern = r'(\w+)\((.*?)\)(?=,\s*\w+\(|$)'
    matches = re.findall(pattern, tool_message)
    
    for func_name, args_str in matches:
        expr = f"{func_name}({args_str})"
        tree = ast.parse(expr, mode='eval')
        call_node = tree.body
        parsed_call = {
            'function': call_node.func.id,
            'arguments': {}
        }
        
        for keyword in call_node.keywords:
            parsed_call['arguments'][keyword.arg] = ast.literal_eval(keyword.value)
        
        calls.append(parsed_call)
    return calls


In [148]:
def infer(messages,tools=None,max_new_tokens=1024,skip_special_tokens=False):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tools = tools,
        return_tensors="pt",
        tokenize=True,
    ).to(model.device)

    output = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.3,
        min_p=0.15,
        repetition_penalty=1.05,
        max_new_tokens=max_new_tokens,
    )

    text = tokenizer.decode(output[0], skip_special_tokens=skip_special_tokens)
    print(text)
    return text.split("<|im_start|>assistant")[-1].strip().replace("<|im_end|>","")

In [149]:

tr = {"read":read}
# messages = [{"role": "user", "content": "What is weather in london and gurgaon?"}]
messages = [
    # {"role":"system","content":"Try to use the available tools and answer user questions."},
    # {"role": "user", "content": "what imports we have in /home/ntlpt59/master/own/agilab/exp/lfm2_claude/main.py file, read and tell me from that file"}
    {"role": "user", "content": "read  /home/ntlpt59/master/own/agilab/exp/lfm2_claude/main.py , use read tool"}
    ]


while True:
    response = infer(messages,tools=list(tr.values()))
    print(response)

    tc = parse_tool_calls(response)
    if not tc: break

    messages.append({'role':'assistant','content':response})

    for etc in tc:
        result = tr[etc['function']](**etc['arguments'])
        messages.append({"role":"tool","content":result})



<|startoftext|><|im_start|>system
List of tools: <|tool_list_start|>[{"type": "function", "function": {"name": "read", "description": "Reads a file from the local filesystem. You can access any file directly by using this tool.\nAssume this tool is able to read all files on the machine. If the User provides a path to a file assume that path is valid. It is okay to read\na file that does not exist; an error will be returned.", "parameters": {"type": "object", "properties": {"path": {"type": "string", "description": "The file path to read from."}}, "required": ["path"]}, "return": {"type": "string", "description": "The file contents as a string."}}}]<|tool_list_end|><|im_end|>
<|im_start|>user
read  /home/ntlpt59/master/own/agilab/exp/lfm2_claude/main.py, use read tool<|im_end|>
<|im_start|>assistant
<|tool_call_start|>[read(path="/home/ntlpt59/master/own/agilab/exp/lfm2_claude/main.py")]<|tool_call_end|><|im_end|>
<|tool_call_start|>[read(path="/home/ntlpt59/master/own/agilab/exp/lfm2_c